# Ta-Feng

## Import packages and define constants

In [1]:
import numpy as np
import pandas as pd
import os
import sys

sys.path.append("../")
sys.path.append("../../")
import random
from pandas.core.frame import DataFrame
import sklearn
from utils.unigramTable import UnigramTable


DEFAULT_USER_COL = "user_ids"
DEFAULT_ITEM_COL = "item_ids"
DEFAULT_ORDER_COL = "order_ids"
DEFAULT_RATING_COL = "ratings"
DEFAULT_LABEL_COL = "label"
DEFAULT_TIMESTAMP_COL = "timestamp"
DEFAULT_PREDICTION_COL = "prediction"
DEFAULT_FLAG_COL = "flag"
data_base_dir = "../../datasets/tafeng/"


test_percents = [0.1, 0.15, 0.2, 0.25, 0.3]
# validate percent just choose the same percent in the training set

negative_size = 100

min_u_c = 10  # items which were purcharsed by at least min_u_c users
min_i_c = 10  # users buy at least min_i_c items
min_o_c = 10  ##users have at least min_o_c orders

/users/tr.zaiqm/mycode/tr_rec/configs/
/users/tr.zaiqm/mycode/tr_rec/datasets/
/users/tr.zaiqm/mycode/tr_rec/checkpoints/
/users/tr.zaiqm/mycode/tr_rec/results/
/users/tr.zaiqm/mycode/tr_rec/samples/
/users/tr.zaiqm/mycode/tr_rec/logs/
/users/tr.zaiqm/mycode/tr_rec/runs/


## load full data to sequence DataFrame

In [2]:
def data_prepro_to_seq(data_base_dir):
    #     if not os.path.isdir(data_base_dir + "data"):  # dir to output preprosessed data
    #         print(
    #             "no data proprecessing dir found, creat dir to output the preprosessed data"
    #         )
    #         os.makedirs(data_base_dir + "data")

    #     print("start loading data from raw data")

    transaction_data = data_base_dir + "transaction_data.csv"
    prior_transaction = pd.read_csv(
        transaction_data,
        usecols=["BASKET_ID", "household_key", "PRODUCT_ID", "DAY", "TRANS_TIME"],
    )

    prior_transaction["DAY"] = prior_transaction["DAY"].astype(str)  #
    prior_transaction["TRANS_TIME"] = prior_transaction["TRANS_TIME"].astype(str)

    prior_transaction["time"] = (
        prior_transaction["DAY"] + prior_transaction["TRANS_TIME"]
    )
    prior_transaction["time"] = prior_transaction["time"].astype(int)  #
    prior_transaction.reset_index(inplace=True)
    prior_transaction = prior_transaction.sort_values(by="time", ascending=False)

    prior_transaction.drop(["DAY", "TRANS_TIME"], axis=1)

    prior_transaction = prior_transaction[
        ["BASKET_ID", "household_key", "PRODUCT_ID", "time"]
    ]
    prior_transaction.insert(3, "flag", "train")
    prior_transaction.insert(4, "ratings", 1)
    prior_transaction.rename(
        columns={
            "BASKET_ID": DEFAULT_ORDER_COL,
            "household_key": DEFAULT_USER_COL,
            "PRODUCT_ID": DEFAULT_ITEM_COL,
            "flag": DEFAULT_FLAG_COL,
            "ratings": DEFAULT_RATING_COL,
            "time": DEFAULT_TIMESTAMP_COL,
        },
        inplace=True,
    )

    print("loading raw data completed")
    return prior_transaction

In [3]:
data_base_dir = "../../datasets/dunnhumby/"
full_data = data_prepro_to_seq(
    data_base_dir + "raw/dunnhumby_The-Complete-Journey/csv/"
)
full_data.head(10)

loading raw data completed


,order_ids,user_ids,item_ids,flag,ratings,timestamp
2592448,42289544807,623,1008673,train,1,7112343
2591966,42289474243,820,18038469,train,1,7112334
2591965,42289474243,820,10149684,train,1,7112334
2591964,42289474243,820,973016,train,1,7112334
2591963,42289474243,820,934131,train,1,7112334
2591962,42289474243,820,911974,train,1,7112334
2591961,42289474243,820,878141,train,1,7112334
2591386,42289363311,1651,1114483,train,1,7112301
2591387,42289363311,1651,9526239,train,1,7112301
2592825,42289604274,2122,949616,train,1,7112259


In [13]:
item_ids = full_data[full_data['order_ids']==26984851472][['user_ids','item_ids']].item_ids.unique()
a = full_data[full_data['order_ids']==26984851472][['user_ids','item_ids']]
b = full_data[full_data['order_ids']==26984851472][['user_ids','item_ids']]

In [27]:
c = pd.merge(a, b, on='user_ids', how='outer')
c[c['item_ids_x']!=c['item_ids_y']]

,user_ids,item_ids_x,item_ids_y
1,2375,1036325,1082185
2,2375,1036325,8160430
3,2375,1036325,1004906
4,2375,1036325,1033142
5,2375,1082185,1036325
7,2375,1082185,8160430
8,2375,1082185,1004906
9,2375,1082185,1033142
10,2375,8160430,1036325
11,2375,8160430,1082185


In [23]:
a.join(b, how='outer', lsuffix='_1', rsuffix='_2')

,user_ids_1,item_ids_1,user_ids_2,item_ids_2
2,2375,1036325,2375,1036325
3,2375,1082185,2375,1082185
4,2375,8160430,2375,8160430
0,2375,1004906,2375,1004906
1,2375,1033142,2375,1033142


In [5]:
full_data_gy = full_data.groupby([DEFAULT_ORDER_COL, DEFAULT_USER_COL])[DEFAULT_ITEM_COL].apply(list).reset_index()
full_data_gy

,order_ids,user_ids,item_ids
0,26984851472,2375,"[1036325, 1082185, 8160430, 1004906, 1033142]"
1,26984851516,2375,"[6423775, 826249, 9487839, 1102651, 1085983, 1..."
2,26984896261,1364,"[981760, 937406, 920955, 897044, 842930]"
3,26984905972,1130,"[833715, 866950, 1022843, 1048462, 1071333]"
4,26984945254,1173,"[824399, 923972, 1131351]"
...,...,...,...
276479,42302712006,2262,"[882441, 1117219, 1135573, 14025651]"
276480,42302712189,1369,"[9526344, 1135096, 959737, 844054, 844759]"
276481,42302712298,2225,"[826144, 1110111, 824005, 904148, 856345, 6442..."
276482,42305362497,1598,"[106589, 17904923, 17295222, 114397, 49659]"


## Row data staticstics

In [4]:
n_interact = len(full_data.index)
n_orders = full_data[DEFAULT_ORDER_COL].nunique()
n_users = full_data[DEFAULT_USER_COL].nunique()
n_items = full_data[DEFAULT_ITEM_COL].nunique()
(n_interact, n_orders, n_users, n_items)

(2595732, 276484, 2500, 92339)

## Save these sequence data to csv files

## load the seqence data from file

In [5]:
full_data.max()

order_ids    42305362535
user_ids            2500
item_ids        18316298
flag               train
ratings                1
timestamp        7112343
dtype: object

### Test the integrity of the saved data

In [6]:
n_interact = len(full_data.index)
n_orders = full_data[DEFAULT_ORDER_COL].nunique()
n_users = full_data[DEFAULT_USER_COL].nunique()
n_items = full_data[DEFAULT_ITEM_COL].nunique()
(n_interact, n_orders, n_users, n_items)

(2595732, 276484, 2500, 92339)

## negative sampling

In [7]:
def feed_neg_sample(eval_df, negative_num, item_sampler):
    print("sampling negative items...")
    interact_status = eval_df.groupby(["user_ids"])["item_ids"].apply(set).reset_index()
    total_interact = pd.DataFrame(
        {"user_ids": [], "item_ids": [], "ratings": []}, dtype=np.long
    )
    for index, user_items in interact_status.iterrows():
        u = int(user_items["user_ids"])
        items = set(user_items["item_ids"])  # item set for user u
        n_items = len(items)  # number of positive item for user u
        sample_neg_items = set(
            item_sampler.sample(negative_num + n_items, 1, True)
        )  # first sample negative_num+n_items items
        sample_neg_items = list(sample_neg_items - items)[:negative_num]
        # filter the positive items and truncate the first negative_num
        #     print(len(sample_neg_items))
        tp_items = np.append(list(items), sample_neg_items)
        #     print(len(tp_items))

        tp_users = np.array([1] * (negative_num + n_items), dtype=np.long) * u
        tp_ones = np.ones(n_items, dtype=np.long)
        tp_zeros = np.zeros(negative_num, dtype=np.long)
        ratings = np.append(tp_ones, tp_zeros)
        #     print(len(tp_users)),print(len(tp_items)),print(len(ratings))
        tp = pd.DataFrame(
            {"user_ids": tp_users, "item_ids": tp_items, "ratings": ratings}
        )
        total_interact = total_interact.append(tp)

    total_interact = sklearn.utils.shuffle(total_interact)
    return total_interact

##  divide data into train, validata and test sets, where validata set is in the train set  

## filter data by count of users, items and orders

In [8]:
# group by group_col and filter filter_col that has less num unique() count
def fiter_by_count(tp, group_col, filter_col, num):
    ordercount = (
        tp.groupby([group_col])[filter_col].nunique().rename("count").reset_index()
    )
    filter_tp = tp[
        tp[group_col].isin(ordercount[ordercount["count"] >= num][group_col])
    ]
    return filter_tp


# filter data by the minimum purcharce number of items and users
def filter_triplets(tp, min_u_c=5, min_i_c=5, min_o_c=5):
    print("filter data by the minimum purcharce number of items and users and orders")
    n_interact = len(tp.index)
    n_orders = tp[DEFAULT_ORDER_COL].nunique()
    n_users = tp[DEFAULT_USER_COL].nunique()
    n_items = tp[DEFAULT_ITEM_COL].nunique()
    print("before filter", n_interact, n_orders, n_users, n_items)
    # Filter users by mixmum number of orders
    if min_o_c > 0:
        tp = fiter_by_count(tp, DEFAULT_USER_COL, DEFAULT_ORDER_COL, min_o_c)

    # Filter users by mixmum number of items
    if min_i_c > 0:
        tp = fiter_by_count(tp, DEFAULT_USER_COL, DEFAULT_ITEM_COL, min_i_c)

    # Filter items by mixmum number of users
    if min_u_c > 0:
        tp = fiter_by_count(tp, DEFAULT_ITEM_COL, DEFAULT_USER_COL, min_u_c)

    n_interact = len(tp.index)
    n_orders = tp[DEFAULT_ORDER_COL].nunique()
    n_users = tp[DEFAULT_USER_COL].nunique()
    n_items = tp[DEFAULT_ITEM_COL].nunique()
    print("after filter", n_interact, n_orders, n_users, n_items)
    # Update both usercount and itemcount after filtering
    # usercount, itemcount = get_count(tp, 'user_ids'), get_count(tp, 'item_ids')
    return tp

In [9]:
def save_train_to_files(data, data_base_dir):

    user_ids = data[DEFAULT_USER_COL].to_numpy(dtype=np.long)
    item_ids = data[DEFAULT_ITEM_COL].to_numpy(dtype=np.long)
    order_ids = data[DEFAULT_ORDER_COL].to_numpy(dtype=np.long)
    timestamps = data[DEFAULT_TIMESTAMP_COL].to_numpy(dtype=np.long)
    ratings = data[DEFAULT_RATING_COL].to_numpy(dtype=np.float32)
    data_file = os.path.join(data_base_dir, "leave_one_basket")
    if not os.path.exists(data_file):
        os.makedirs(data_file)
    data_file = os.path.join(data_file, "train.npz")
    np.savez_compressed(
        data_file,
        user_ids=user_ids,
        item_ids=item_ids,
        order_ids=order_ids,
        timestamp=timestamps,
        ratings=ratings,
    )
    print(
        "Data saving to file:",
        data_base_dir,
        "max_item_num:",
        np.max(item_ids),
        "max_user_num:",
        np.max(user_ids),
    )

In [10]:
def save_test_to_files(data, data_base_dir, suffix):
    user_ids = data[DEFAULT_USER_COL].to_numpy(dtype=np.long)
    item_ids = data[DEFAULT_ITEM_COL].to_numpy(dtype=np.long)
    ratings = data[DEFAULT_RATING_COL].to_numpy(dtype=np.float32)
    data_file = os.path.join(data_base_dir, "leave_one_basket")
    if not os.path.exists(data_file):
        os.makedirs(data_file)
    data_file = os.path.join(data_file, suffix)
    np.savez_compressed(
        data_file, user_ids=user_ids, item_ids=item_ids, ratings=ratings,
    )
    print(
        "Data saving to file:",
        data_base_dir,
        "max_item_num:",
        np.max(item_ids),
        "max_user_num:",
        np.max(user_ids),
    )

In [11]:
full_data.head()

,order_ids,user_ids,item_ids,flag,ratings,timestamp
2592448,42289544807,623,1008673,train,1,7112343
2591966,42289474243,820,18038469,train,1,7112334
2591965,42289474243,820,10149684,train,1,7112334
2591964,42289474243,820,973016,train,1,7112334
2591963,42289474243,820,934131,train,1,7112334


In [12]:
print("leave_one_basket_split")
from tqdm import tqdm

fiter_full_data = fiter_by_count(full_data, DEFAULT_ITEM_COL, DEFAULT_USER_COL, 10)
fiter_full_data = fiter_by_count(
    fiter_full_data, DEFAULT_USER_COL, DEFAULT_ORDER_COL, 3
)
users = fiter_full_data[DEFAULT_USER_COL].unique()
for user in tqdm(users):
    user_data_orders = fiter_full_data[
        fiter_full_data[DEFAULT_USER_COL] == user
    ].sort_values(by=["timestamp"], ascending=False)
    unique_user_orders = user_data_orders[DEFAULT_ORDER_COL].unique()
    fiter_full_data.loc[
        fiter_full_data[DEFAULT_ORDER_COL] == unique_user_orders[0], DEFAULT_FLAG_COL,
    ] = "test"
    fiter_full_data.loc[
        fiter_full_data[DEFAULT_ORDER_COL] == unique_user_orders[1], DEFAULT_FLAG_COL,
    ] = "validate"

leave_one_basket_split


100%|██████████| 2486/2486 [18:54<00:00,  2.19it/s]


In [13]:
print("sampling negatives and saving")
data_base_dir = "../../datasets/dunnhumby/"

tp_train = fiter_full_data[fiter_full_data[DEFAULT_FLAG_COL] == "train"]
tp_validate = fiter_full_data[fiter_full_data[DEFAULT_FLAG_COL] == "validate"]
tp_test = fiter_full_data[fiter_full_data[DEFAULT_FLAG_COL] == "test"]

save_train_to_files(tp_train, data_base_dir)
item_sampler = UnigramTable(tp_train[DEFAULT_ITEM_COL].value_counts().to_dict())
for i in range(10):
    tp_validate_new = feed_neg_sample(tp_validate, 100, item_sampler)
    tp_test_new = feed_neg_sample(tp_test, 100, item_sampler)
    save_test_to_files(tp_validate_new, data_base_dir, suffix="valid" + "_" + str(i))
    save_test_to_files(tp_test_new, data_base_dir, suffix="test" + "_" + str(i))

sampling negatives and saving
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
Filling unigram table
sampling negative items...
sampling negative items...
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
sampling negative items...
sampling negative items...
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
sampling negative items...
sampling negative items...
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
sampling negative items...
sampling negative items...
Data saving to file: ../../datasets/dunnhumby/ max_item_num: 18203921 max_user_num: 2500
Data saving to file:

In [14]:
len(tp_train.index), len(tp_validate.index), len(tp_test.index)

(2330466, 26610, 26951)

In [15]:
tp_train.DEFAULT_USER_COL.nunique(), tp_train.DEFAULT_ITEM_COL.nunique()

AttributeError: 'DataFrame' object has no attribute 'DEFAULT_USER_COL'

In [18]:
tp_train.user_ids.nunique(), tp_train.item_ids.nunique()

(2486, 23404)